# Scikit Learn and the K-nearest Neighbor Algorithm

In this notebook we'll introduce the `sklearn` package and a few important concepts in machine learning:

* Splitting data into test, train, and validation sets.
* Fitting models to a dataset.
* And using "Hyperparameters" to tune models. 

Lets revisit the example we saw in the first class:

In [4]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

# Load the data
heart_dataset = pd.read_csv('../../datasets/uci-heart-disease/heart.csv')

# Split the data into input and labels
labels = heart_dataset['target']
input_data = heart_dataset.drop(columns=['target'])

# Split the data into training and test
training_data, test_data, training_labels, test_labels = train_test_split(
    input_data, 
    labels, 
    test_size=0.20
)

# Build the model
model = KNeighborsClassifier()
model.fit(training_data, training_labels)

# See how it did.
print("Test accuracy: ", model.score(test_data, test_labels))

Test accuracy:  0.6557377049180327


# Test/Train/Validation Splits:

In machine learning, it's important to avoid something called "overfitting" our models. This happens when a model more or less "memorizes" the training data, but performs poorly on data that is "held out" of the training process. A model that is "overfit" won't "generalize" to unseen data — making it useless in the real world!

To avoid and detect overfitting we will commonly split our data into 3 buckets:

* Training data: the data that the model actually learns from.
* Validation data: data that the model doesn't learn from, but that we use to validate the results throughout the process of building our models.
* Test data: data that is held out entierly during the model building process in order to give an unbiased measure of the final models performance. 
    * If we use the test data, and then change our model, the test data is no longer "unbiased" as we will have incorporated information from the test data (i.e. our models poor "test performance") into the next version of the model. In this case we have turned the test data into validation data, and we should get new test data.

In the above example we've only made two buckets of data, which we called training and test...  In theory we COULD do this:

In [5]:
# Split the data into training+validation and test
training_and_validation_data, test_data, training_and_validation_labels, test_labels = train_test_split(
    input_data, 
    labels, 
    test_size=0.20
)

# Split the training+validation data into training and validation
training_data, validation_data, training_labels, validation_labels = train_test_split(
    training_and_validation_data, 
    training_and_validation_labels, 
    test_size=0.20
)

# Then use only training and validation to evaluate our model and make changes to the model's performance...

While the above CAN be done, it's much more common to make the test/train split much earlier in the data pipeline. Many datasets for ML tasks come with a test set already prepared and separate from the training set. ML practitioners then perform a training/validation split with all of the training data. The training and test data will be saved separately, in their own folders or CSV files or labeled differently in the database/data warehouse software.

If you've collected your own data and want to do ML with it, I strongly suggest you split out 10%-20% of that data, set it aside, and don't look at it until the very end of your ML pipeline to get an unbiased evaluation once you've built a model you like. 

# SKLearn's API

Scikit learn has a wonderfully unified API that always follows this pattern: 

* Create a model from a class.
    * This is where you set the "hyperparameters" of the model.
* Call that model's `.fit` method using the training data to train the model.
* Call that model's `.score` method to evaluate the model against the validation/test data.

For example:

In [12]:
# Lets build multiple models using a few different "hyperparameters"
model_one = KNeighborsClassifier()
model_two = KNeighborsClassifier(weights='distance')
model_three = KNeighborsClassifier(n_neighbors=10, weights='distance')

for i, model in enumerate([model_one, model_two, model_three]):
    model.fit(training_data, training_labels)

    print(f' {i+1} validation accuracy: ', model.score(validation_data, validation_labels))

 1 validation accuracy:  0.6530612244897959
 2 validation accuracy:  0.6530612244897959
 3 validation accuracy:  0.673469387755102


# The K-Nearest Neighbor's Model

So what is the actual difference between these three models? How does KNN actually work?

KNN is a relatively straightforward model. When you want to make a prediction with KNN you simply compare the item you're making a prediction about to the training dataset using a distance function and based on the class of the "nearest" neighbors the model makes a prediction.

K is how many neighbors to look at, if k is 5 the model looks at the 5 nearest neighbors and whichever class is most common among those 5 neighbors is the one selected. Lets look at some pictures from the pre-reading (https://towardsdatascience.com/laymans-introduction-to-knn-c793ed392bc2):

![](https://miro.medium.com/max/552/1*6YK2xQ4wxBGGrCaegT9JfA.png)

![](https://miro.medium.com/max/552/1*z-y9I2aHAGj4GtMI5cR1OA.png)

![](https://miro.medium.com/max/552/1*7tSKxmXPca1IlgjRHtwOGg.png)

![](https://miro.medium.com/max/552/1*_EYdoVX941aZXa5BH6XnHQ.png)

These examples are all in 2-dimensional space, but the algorithm generalizes to n-dimensions (based on the number of features in our training data). 

K is controlled in `sklearn` by the `n_neighbors` parameter. 

Another hyperparameter in KNN is the `weights` parameter, which has 3 possible values, from the docs (https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html):

* ‘uniform’ : uniform weights. All points in each neighborhood are weighted equally.
* ‘distance’ : weight points by the inverse of their distance. in this case, closer neighbors of a query point will have a greater influence than neighbors which are further away.
* [callable] : a user-defined function which accepts an array of distances, and returns an array of the same shape containing the weights.

Similarly, the distance metric can be provided:

> metric: str or callable, default=’minkowski’

> the distance metric to use for the tree. The default metric is minkowski, and with p=2 is equivalent to the standard Euclidean metric. See the documentation of DistanceMetric for a list of available metrics. If metric is “precomputed”, X is assumed to be a distance matrix and must be square during fit. X may be a sparse graph, in which case only “nonzero” elements may be considered neighbors.
